In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

from py.is_rebalance import is_rebalance
from py.signals import signals
from py.split_df import split_df
from py.simulate import simulate

In [8]:
# VARIABLES

# We want 180 day windows (6 months) with 120 day overlap (4 months)
# Since our dataframe is in hours, multiply by 24
window_len = 24 * 180
overlap = 24 * 120

# Assets traded
assets = ['ETH', 'USD']

# Moving average intervals used
moving_averages = [50, 100, 200]

# Potential ETH to DAI allocations from bullish signals
bull_allocation = [
    [0.85, 0.15],
    [0.80, 0.20],
    [0.75, 0.25],
    [0.70, 0.30],
    [0.65, 0.35],
]

# List of allocations used, with the inverse allocation for bearish signals
allocation_lst = [{'bull': b,
                   'neutral': [0.50, 0.50],
                   'bear': b[::-1]}
                  for b in bull_allocation]


# Minimum difference in weighting needed to rebalance without a new signal
# This prevents unnecessary rebalancing
wiggle_room_lst = np.arange(0, 0.11, 0.01)

In [9]:
df = pd.read_csv('../data/ETH-USDT.csv', usecols=['date', 'close']).rename({'close':'ETH'}, axis=1)
df['date'] = pd.to_datetime(df['date'])
df['USD'] = 1

# Create columns to 
df['rebalance'] = is_rebalance(df['date'])
df['signal'] = signals(df['ETH'], df['rebalance'], *moving_averages)

# Split dataframe into windows 
dfs = split_df(df.to_dict(orient='records'), overlap, window_len)

results = []

for allocation in allocation_lst:
    for wiggle_room in wiggle_room_lst:
        result = {
            'wiggle_room': wiggle_room,
            'allocation': '/'.join(str(x) for x in allocation['bull']),
        }

        # Add result for each split dataframe
        for df_split in dfs:
            start = datetime.strftime(df_split[0]['date'], '%Y.%m.%d')
            end = datetime.strftime(df_split[-1]['date'], '%Y.%m.%d')

            _, _, performance = simulate(assets, allocation, wiggle_room, df_split)

            result[start + '-' + end] = performance

        # Save result to results
        results.append(result)


# Convert dict to dataframe
df_results = pd.DataFrame.from_records(results)
df_results['sum'] = df_results.drop(['wiggle_room', 'allocation'], axis=1).sum(axis=1)

# Sort
df_results = df_results.sort_values('sum', ascending=False)


# Save signals and performance to CSV
df.to_csv('backtests/signals.csv', index=False)
df_results.to_csv('backtests/performance.csv', index=False)

In [11]:
df_results

,wiggle_room,allocation,2017.08.16-2018.02.14,2017.10.16-2018.04.15,2017.12.15-2018.06.14,2018.02.14-2018.08.14,2018.04.15-2018.10.13,2018.06.14-2018.12.12,2018.08.14-2019.02.10,2018.10.13-2019.04.11,2018.12.12-2019.06.11,2019.02.10-2019.08.10,2019.04.11-2019.10.09,2019.06.11-2019.12.08,2019.08.09-2020.02.06,sum
10,0.10,0.85/0.15,-736.237384,-411.724915,721.889748,1999.663720,1181.867113,2322.855536,1720.834231,1531.082805,-1013.944871,-869.830811,-121.778869,786.093514,692.442656,7803.212475
8,0.08,0.85/0.15,-729.453268,-372.128274,663.238996,1982.533402,1155.055067,2335.277395,1854.711006,1551.490892,-1054.661190,-831.093177,-89.634775,765.426239,556.893984,7787.656296
9,0.09,0.85/0.15,-736.237384,-425.388094,539.084673,1999.663720,1182.915433,2373.511398,1854.711006,1531.082805,-1004.543371,-828.978647,-81.279580,765.525218,556.893984,7726.961161
7,0.07,0.85/0.15,-778.033001,-372.128274,663.238996,1982.533402,1152.518427,2326.446238,1716.635597,1551.490892,-1064.869437,-831.093177,-89.634775,765.426239,549.634659,7572.165783
6,0.06,0.85/0.15,-774.393565,-372.581631,653.217933,2004.859653,1167.754629,2312.891394,1689.908421,1551.490892,-1079.527676,-831.093177,-89.634775,741.733749,549.634659,7524.260504
21,0.10,0.8/0.2,-651.465115,-352.861123,635.241140,1778.929373,1160.993047,2329.362063,1753.165916,1494.075449,-955.861653,-879.442338,-240.361327,745.790384,642.701542,7460.267359
20,0.09,0.8/0.2,-679.528391,-442.962487,635.241140,1925.548558,1162.067309,2297.615864,1679.347548,1471.447430,-991.118007,-865.693599,-199.480938,765.582897,678.283442,7436.350768
5,0.05,0.85/0.15,-765.971596,-372.581631,651.251942,1986.119273,1123.620121,2248.005210,1730.594284,1539.135059,-1079.664228,-834.400307,-99.456061,741.733749,549.634659,7418.020473
19,0.08,0.8/0.2,-716.936990,-453.776046,477.279670,1925.548558,1142.956439,2301.950516,1789.574775,1471.447430,-983.303517,-838.616279,-136.931358,752.974135,639.398055,7371.565387
18,0.07,0.8/0.2,-716.936990,-426.436906,543.829233,1910.394760,1111.704818,2266.308575,1789.574775,1484.953416,-1030.850353,-833.961518,-135.033675,736.841251,564.721101,7265.108487
